In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark


In [30]:
df1 = spark.createDataFrame([('2022-09-10 3:12:59',)], ['DOB'])
df1.show()
from pyspark.sql.functions import to_date, to_timestamp
df1.select(df1.DOB, to_date(df1.DOB)).show()
df1.withColumn("Age",datediff(current_date, to_date)/365).show()
#df1.show()

+------------------+
|               DOB|
+------------------+
|2022-09-10 3:12:59|
+------------------+

+------------------+------------+
|               DOB|to_date(DOB)|
+------------------+------------+
|2022-09-10 3:12:59|  2022-09-10|
+------------------+------------+



NameError: ignored

In [55]:
emp = [(1, "A", "2018", "M", "10", 30000), (2, "B", "2010", "M", "30", 40000), (3, "C", "2010", "M", "20", 10000), \
       (4, "D", "2005", "F", "40", 20000), (5, "E", "2010", "M", "20", 12000), (6, "F", "2010", "M", "10", 28000), \
       (7, "G", "2015", "M", "40", 11000), (8, "H", "2010", "F", "30", 22000), (9, "I", "2012", "F", "40", 32000),\
       (10, "J", "2010", "F", "30", 42000)
  ]
empColumns = ["emp_id","name","year_joined", \
       "gender","emp_dept_id","salary"]

empDF = spark.createDataFrame(data=emp, schema = empColumns)
empDF.printSchema()
empDF.show()

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- salary: long (nullable = true)

+------+----+-----------+------+-----------+------+
|emp_id|name|year_joined|gender|emp_dept_id|salary|
+------+----+-----------+------+-----------+------+
|     1|   A|       2018|     M|         10| 30000|
|     2|   B|       2010|     M|         30| 40000|
|     3|   C|       2010|     M|         20| 10000|
|     4|   D|       2005|     F|         40| 20000|
|     5|   E|       2010|     M|         20| 12000|
|     6|   F|       2010|     M|         10| 28000|
|     7|   G|       2015|     M|         40| 11000|
|     8|   H|       2010|     F|         30| 22000|
|     9|   I|       2012|     F|         40| 32000|
|    10|   J|       2010|     F|         30| 42000|
+------+----+-----------+------+-----------+------+



In [57]:
dept = [("Finance",10), ("Marketing",20), ("Sales",30), ("IT",40)]
deptColumns = ["dept_name","dept_id"]
deptDF = spark.createDataFrame(data=dept, schema = deptColumns)
deptDF.printSchema()
deptDF.show()

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|  Finance|     10|
|Marketing|     20|
|    Sales|     30|
|       IT|     40|
+---------+-------+



In [58]:
# Inner Join

empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"inner").show()

+------+----+-----------+------+-----------+------+---------+-------+
|emp_id|name|year_joined|gender|emp_dept_id|salary|dept_name|dept_id|
+------+----+-----------+------+-----------+------+---------+-------+
|     1|   A|       2018|     M|         10| 30000|  Finance|     10|
|     6|   F|       2010|     M|         10| 28000|  Finance|     10|
|     2|   B|       2010|     M|         30| 40000|    Sales|     30|
|     8|   H|       2010|     F|         30| 22000|    Sales|     30|
|    10|   J|       2010|     F|         30| 42000|    Sales|     30|
|     3|   C|       2010|     M|         20| 10000|Marketing|     20|
|     5|   E|       2010|     M|         20| 12000|Marketing|     20|
|     4|   D|       2005|     F|         40| 20000|       IT|     40|
|     7|   G|       2015|     M|         40| 11000|       IT|     40|
|     9|   I|       2012|     F|         40| 32000|       IT|     40|
+------+----+-----------+------+-----------+------+---------+-------+



In [59]:
# Outer Join

empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"outer").show()

+------+----+-----------+------+-----------+------+---------+-------+
|emp_id|name|year_joined|gender|emp_dept_id|salary|dept_name|dept_id|
+------+----+-----------+------+-----------+------+---------+-------+
|     1|   A|       2018|     M|         10| 30000|  Finance|     10|
|     6|   F|       2010|     M|         10| 28000|  Finance|     10|
|     2|   B|       2010|     M|         30| 40000|    Sales|     30|
|     8|   H|       2010|     F|         30| 22000|    Sales|     30|
|    10|   J|       2010|     F|         30| 42000|    Sales|     30|
|     3|   C|       2010|     M|         20| 10000|Marketing|     20|
|     5|   E|       2010|     M|         20| 12000|Marketing|     20|
|     4|   D|       2005|     F|         40| 20000|       IT|     40|
|     7|   G|       2015|     M|         40| 11000|       IT|     40|
|     9|   I|       2012|     F|         40| 32000|       IT|     40|
+------+----+-----------+------+-----------+------+---------+-------+



In [60]:
# Full Outer Join

empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"fullouter").show()

+------+----+-----------+------+-----------+------+---------+-------+
|emp_id|name|year_joined|gender|emp_dept_id|salary|dept_name|dept_id|
+------+----+-----------+------+-----------+------+---------+-------+
|     1|   A|       2018|     M|         10| 30000|  Finance|     10|
|     6|   F|       2010|     M|         10| 28000|  Finance|     10|
|     2|   B|       2010|     M|         30| 40000|    Sales|     30|
|     8|   H|       2010|     F|         30| 22000|    Sales|     30|
|    10|   J|       2010|     F|         30| 42000|    Sales|     30|
|     3|   C|       2010|     M|         20| 10000|Marketing|     20|
|     5|   E|       2010|     M|         20| 12000|Marketing|     20|
|     4|   D|       2005|     F|         40| 20000|       IT|     40|
|     7|   G|       2015|     M|         40| 11000|       IT|     40|
|     9|   I|       2012|     F|         40| 32000|       IT|     40|
+------+----+-----------+------+-----------+------+---------+-------+



In [61]:
# Left Outer Join

empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"leftouter").show()

+------+----+-----------+------+-----------+------+---------+-------+
|emp_id|name|year_joined|gender|emp_dept_id|salary|dept_name|dept_id|
+------+----+-----------+------+-----------+------+---------+-------+
|     1|   A|       2018|     M|         10| 30000|  Finance|     10|
|     6|   F|       2010|     M|         10| 28000|  Finance|     10|
|     2|   B|       2010|     M|         30| 40000|    Sales|     30|
|     8|   H|       2010|     F|         30| 22000|    Sales|     30|
|    10|   J|       2010|     F|         30| 42000|    Sales|     30|
|     3|   C|       2010|     M|         20| 10000|Marketing|     20|
|     5|   E|       2010|     M|         20| 12000|Marketing|     20|
|     4|   D|       2005|     F|         40| 20000|       IT|     40|
|     7|   G|       2015|     M|         40| 11000|       IT|     40|
|     9|   I|       2012|     F|         40| 32000|       IT|     40|
+------+----+-----------+------+-----------+------+---------+-------+



In [62]:
# Right Outer Join

empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"rightouter").show()

+------+----+-----------+------+-----------+------+---------+-------+
|emp_id|name|year_joined|gender|emp_dept_id|salary|dept_name|dept_id|
+------+----+-----------+------+-----------+------+---------+-------+
|     1|   A|       2018|     M|         10| 30000|  Finance|     10|
|     6|   F|       2010|     M|         10| 28000|  Finance|     10|
|     2|   B|       2010|     M|         30| 40000|    Sales|     30|
|     8|   H|       2010|     F|         30| 22000|    Sales|     30|
|    10|   J|       2010|     F|         30| 42000|    Sales|     30|
|     3|   C|       2010|     M|         20| 10000|Marketing|     20|
|     5|   E|       2010|     M|         20| 12000|Marketing|     20|
|     4|   D|       2005|     F|         40| 20000|       IT|     40|
|     7|   G|       2015|     M|         40| 11000|       IT|     40|
|     9|   I|       2012|     F|         40| 32000|       IT|     40|
+------+----+-----------+------+-----------+------+---------+-------+

